# Peak shaving vs TCIPC analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.interpolate import griddata
from weis.visualization.utils import load_OMsql_multi
from cmcrameri import cm

plt.style.use("torque.mplstyle")

%matplotlib widget

In [ ]:
# Define the logs to load and give them labels.
logs_to_load = {
    "Free yaw": "../../../data/torque/ps_vs_tcipc/log_ps_vs_tcipc.sql*",
    "Zero yaw": "../../../data/torque/ps_vs_tcipc_zero_yaw/log_ps_vs_tcipc_zero_yaw.sql*",
    "Free yaw we0": "../../../data/torque/ps_vs_tcipc_we0/log_ps_vs_tcipc_we0.sql*",
    "Zero yaw we0": "../../../data/torque/ps_vs_tcipc_zero_yaw_we0/log_ps_vs_tcipc_zero_yaw_we0.sql*",
}

# Load all datasets
all_data_dicts = {}
for log_name, log_fmt in logs_to_load.items():
    all_data_dicts[log_name] = load_OMsql_multi(log_fmt)
    print(f"Loaded {log_name}: {all_data_dicts[log_name].keys()}")

In [ ]:
# Let's define how we load, scale, and label the data, then make a dataframe.
max_pitch_rate_degps = np.rad2deg(0.035)
all_outputs = {
    "rank": {
        "key": "rank",
        "scaling": lambda x: x,
        "label": "rank",
    },
    "iter": {
        "key": "iter",
        "scaling": lambda x: x,
        "label": "iter",
    },
    # ROSCO variables.
    "TCIPC_MaxTipDeflection": {
        "key": "tune_rosco_ivc.TCIPC_MaxTipDeflection",
        "scaling": lambda x: x,
        "label": "TCIPC reference (m)",
    },
    "ps_percent": {
        "key": "tune_rosco_ivc.ps_percent",
        "scaling": lambda x: x,
        "label": "Peak Shaving (-)",
    },
    "peak_shaving": {
        "key": "tune_rosco_ivc.ps_percent",
        "scaling": lambda x: 100 - 100 * x,
        "label": "Peak shaving (%)",
    },
    "TCIPC_nHarmonics": {
        "key": "tune_rosco_ivc.TCIPC_nHarmonics",
        "scaling": lambda x: x,
        "label": "Number of harmonics",
    },
    "TCIPC_ZeroYawDeflection": {
        "key": "tune_rosco_ivc.TCIPC_ZeroYawDeflection",
        "scaling": lambda x: x,
        "label": "Zero yaw deflection",
    },
    # Objectives / responses
    "aep": {
        "key": "aeroelastic.AEP",
        "scaling": lambda x: 1e-6 * x,
        "label": "AEP (GWh)",
    },
    "Ct": {
        "key": "aeroelastic.Ct_out",
        "scaling": lambda x: np.mean(x),
        "label": "Ct (-)",
    },
    "max_TipDxc_towerPassing": {
        "key": "aeroelastic.max_TipDxc_towerPassing",
        "scaling": lambda x: x,
        "label": "Max TipDxc Tower Passing (m)",
    },
    "tower_clearance": {
        "key": "aeroelastic.max_TipDxc_towerPassing",
        "scaling": lambda x: 30 - x,
        "label": "Tower clearance (m)",
    },
    "TCIPC_amplitude_at_max_deflection": {
        "key": "aeroelastic.TCIPC_amplitude_at_max_deflection",
        "scaling": lambda x: x,
        "label": "TCIPC amplitude (deg)",
    },
    "avg_pitch_travel": {
        "key": "aeroelastic.avg_pitch_travel",
        "scaling": lambda x: x / max_pitch_rate_degps,
        "label": "ADC (-)",
    },
    "DEL_RootMyb": {
        "key": "aeroelastic.DEL_RootMyb",
        "scaling": lambda x: x / 1000,
        "label": "Flapping DEL (MNm)",
    },
    "max_TwrBsMyt": {
        "key": "aeroelastic.max_TwrBsMyt",
        "scaling": lambda x: x / 1000,
        "label": "Max Tower Base Myt (MNm)",
    },
}

# Build dataframe from mapping for each log
labels = {short: info["label"] for short, info in all_outputs.items()}
all_dfs = []

for log_name, data_dict in all_data_dicts.items():
    df_dict = {}
    for short_label, info in all_outputs.items():
        data = data_dict[info["key"]]
        scaled_data = list(map(info["scaling"], data))
        df_dict[short_label] = scaled_data

    df_temp = pd.DataFrame(df_dict)
    df_temp["log_name"] = log_name  # Add identifier column
    all_dfs.append(df_temp)

# Combine all dataframes
df = pd.concat(all_dfs, ignore_index=True)
print(f"Combined dataframe shape: {df.shape}")
df.head()

## Data exploration

In [ ]:
# Make a plot of the distribution of our design variables.
plt.figure()
sns.scatterplot(
    df,
    x="TCIPC_MaxTipDeflection",
    y="ps_percent",
    style="log_name",
)
plt.show()

In [ ]:
# Plot several outputs/objectives as a function of the design variables.
# Output variables to plot.
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
    "Ct",
]

fig, axs = plt.subplots(len(logs_to_load), len(outputs), figsize=(15, 5))

for i, log_name in enumerate(logs_to_load.keys()):
    for j, output in enumerate(outputs):
        scatter = axs[i, j].scatter(
            df[df["log_name"] == log_name]["peak_shaving"],
            df[df["log_name"] == log_name]["TCIPC_MaxTipDeflection"],
            c=df[df["log_name"] == log_name][output],
        )

        axs[i, j].set_xlabel("Peak shaving (-)")
        axs[i, j].set_ylabel("TCIPC max tip deflection (m)")
        axs[i, j].set_title(output)
        plt.colorbar(scatter, ax=axs[i, j])

    print(log_name)

plt.tight_layout()
plt.show()

In [ ]:
# Get an idea of the trade-off between AEP and tower clearance.
fig, ax = plt.subplots()

df.sort_values("aep", inplace=True)

sns.lineplot(
    data=df[df["TCIPC_MaxTipDeflection"] == 0.0],
    x="aep",
    y="tower_clearance",
    hue="log_name",
)
sns.lineplot(
    data=df[df["TCIPC_MaxTipDeflection"] == 20.0],
    x="aep",
    y="tower_clearance",
    hue="log_name",
    palette="Reds",
)

## Data interpolation

In [ ]:
# Define the bounds of our design space for interpolation.
ps_min, ps_max = (
    df["peak_shaving"].min(),
    df["peak_shaving"].max(),
)
tip_min, tip_max = (
    df["TCIPC_MaxTipDeflection"].min(),
    df["TCIPC_MaxTipDeflection"].max(),
)

# Create a regular grid for interpolation to enable smooth contour plots.
n_points = 200
ps_grid = np.linspace(ps_min, ps_max, n_points)
tip_grid = np.linspace(tip_min, tip_max, n_points)
peak_shaving_grid, tcipc_reference_grid = np.meshgrid(ps_grid, tip_grid)

# Interpolate each output variable on the grid for each log.
# We store the results in a nested dictionary for easy access when plotting.
interpolated_data = {}

for log_name in logs_to_load.keys():
    interpolated_data[log_name] = {}
    df_log = df[df["log_name"] == log_name]

    # Extract the independent variables as points for interpolation.
    points = df_log[["peak_shaving", "TCIPC_MaxTipDeflection"]].values

    for output in outputs:
        # Extract the dependent variable values.
        values = df_log[output].values

        # Interpolate using linear method, which works well for scattered data.
        grid_values = griddata(
            points,
            values,
            (peak_shaving_grid, tcipc_reference_grid),
            method="cubic",
        )

        interpolated_data[log_name][output] = grid_values

print(f"Interpolated {len(outputs)} outputs for {len(logs_to_load)} datasets")
print(f"Grid shape: {peak_shaving_grid.shape}")

In [ ]:
# Plot several outputs/objectives as a function of the design variables.
# Output variables to plot.
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
    "Ct",
]
clims = [
    [70, 95],
    [10, 30],
    [0, 1],
    [0, 5],
    [15, 50],
    [150, 400],
    [0.44, 0.6],
]

fig, axs = plt.subplots(len(logs_to_load), len(outputs), figsize=(12, 6))

for i, log_name in enumerate(logs_to_load.keys()):
    print(log_name)
    for j, output in enumerate(outputs):
        levels = np.linspace(clims[j][0], clims[j][1], 9)
        scatter = axs[i, j].contourf(
            peak_shaving_grid,
            tcipc_reference_grid,
            interpolated_data[log_name][output],
            levels=levels,
            vmin=clims[j][0],
            vmax=clims[j][1],
        )

        axs[i, j].set_xlabel("Peak shaving (-)")
        axs[i, j].set_ylabel("TCIPC max tip deflection (m)")
        axs[i, j].set_title(output)
        plt.colorbar(scatter, ax=axs[i, j])

plt.tight_layout()

## Plots for Torque

In [ ]:
plt.style.use("torque.mplstyle")

In [ ]:
df.groupby("log_name").max()

In [ ]:
df.groupby("log_name").min()

In [ ]:
# Plot 1: Parameter sweep for free yaw with contour lines for TCIPC amplitude.
outputs_plot = ["tower_clearance", "aep", "avg_pitch_travel", "DEL_RootMyb"]
clims_plot = [
    [10, 24],
    [70, 91],
    [0, 0.7],
    [20, 50],
]

fig, axs = plt.subplots(
    1, len(outputs_plot), figsize=(plt.rcParams["figure.figsize"][0], 1.75)
)

log_name = "Free yaw"

for j, output in enumerate(outputs_plot):
    levels = np.linspace(clims_plot[j][0], clims_plot[j][1], 13)
    contourf = axs[j].contourf(
        peak_shaving_grid,
        tcipc_reference_grid,
        interpolated_data[log_name][output],
        levels=levels,
        vmin=clims_plot[j][0],
        vmax=clims_plot[j][1],
    )

    # Add contour lines for TCIPC amplitude.
    contour_inactive = axs[j].contour(
        peak_shaving_grid,
        tcipc_reference_grid,
        interpolated_data[log_name]["TCIPC_amplitude_at_max_deflection"],
        levels=[0.25],
        colors="black",
        linestyles="--",
    )
    contour_saturated = axs[j].contour(
        peak_shaving_grid,
        tcipc_reference_grid,
        interpolated_data[log_name]["TCIPC_amplitude_at_max_deflection"],
        levels=[4.5],
        colors="black",
        linestyles="-",
    )

    # Add labels to contour lines.
    axs[j].clabel(contour_inactive, inline=True, fontsize=8, fmt=" inactive ")
    axs[j].clabel(contour_saturated, inline=True, fontsize=8, fmt=" saturated ")

    axs[j].set_xlabel("Peak shaving (%)")
    if j == 0:
        axs[j].set_ylabel("TCIPC reference (m)")
    axs[j].set_yticks([0, 5, 10, 15, 20])
    axs[j].set_xticks([0, 25, 50])
    axs[j].set_title(labels[output])
    cbar = plt.colorbar(contourf, ax=axs[j])
    cbar.ax.tick_params(labelsize=8)
    cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x:.1f}"))

plt.tight_layout(pad=0.1, w_pad=0.5)

plt.savefig("../figures/parameter_sweep.pdf")
plt.show()

In [ ]:
# Plot 2: Absolute difference between zero yaw and free yaw.
fig, axs = plt.subplots(
    1, len(outputs_plot), figsize=(plt.rcParams["figure.figsize"][0], 1.75)
)

# Compute the absolute differences for each output.
diff_data = {}
for output in outputs_plot:
    diff_data[output] = (
        (interpolated_data["Zero yaw"][output] - interpolated_data["Free yaw"][output])
        / interpolated_data["Free yaw"][output]
        * 100
    )

clims_diff = [
    [-15, 15],
    [-15, 15],
    [-180, 180],
    [-15, 15],
]
# clims_diff = [[-20, 20]] * len(outputs_plot)

for j, output in enumerate(outputs_plot):
    levels = np.linspace(clims_diff[j][0], clims_diff[j][1], 13)
    contourf = axs[j].contourf(
        peak_shaving_grid,
        tcipc_reference_grid,
        diff_data[output],
        levels=levels,
        vmin=clims_diff[j][0],
        vmax=clims_diff[j][1],
        cmap=cm.cork,
        # extend="both",
    )

    # Add contour lines for TCIPC amplitude from free yaw.
    contour_inactive = axs[j].contour(
        peak_shaving_grid,
        tcipc_reference_grid,
        interpolated_data["Free yaw"]["TCIPC_amplitude_at_max_deflection"],
        levels=[0.5],
        colors="black",
        linestyles="--",
    )
    contour_saturated = axs[j].contour(
        peak_shaving_grid,
        tcipc_reference_grid,
        interpolated_data["Free yaw"]["TCIPC_amplitude_at_max_deflection"],
        levels=[4.5],
        colors="black",
        linestyles="-",
    )

    # Add labels to contour lines.
    axs[j].clabel(contour_inactive, inline=True, fontsize=8, fmt=" inactive ")
    axs[j].clabel(contour_saturated, inline=True, fontsize=8, fmt=" saturated ")

    axs[j].set_xlabel("Peak shaving (%)")
    if j == 0:
        axs[j].set_ylabel("TCIPC reference (m)")
    axs[j].set_yticks([0, 5, 10, 15, 20])
    axs[j].set_xticks([0, 25, 50])
    label_text = labels[output].rsplit("(", 1)[0].strip()
    axs[j].set_title(f"Δ {label_text} (%)")
    cbar = plt.colorbar(contourf, ax=axs[j])
    cbar.ax.tick_params(labelsize=8)
    cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x:.1f}"))

plt.tight_layout(pad=0.1, w_pad=0.5)

plt.savefig("../figures/parameter_sweep_zero_yaw_difference.pdf")
plt.show()

In [ ]:
from scipy.interpolate import CloughTocher2DInterpolator

In [ ]:
# Plot 3: Pareto front (trade-off curve).

# Create separate interpolators for each log
interpolators = {}
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
]
signs = [-1, -1, +1, +1, +1, +1]
for log_name in all_data_dicts.keys():
    df_log = df[df["log_name"] == log_name]
    xy = df_log[["ps_percent", "TCIPC_MaxTipDeflection"]].values

    interpolators[log_name] = {}

    for j, output in enumerate(outputs):
        interpolators[log_name][output] = CloughTocher2DInterpolator(
            xy, signs[j] * df_log[output].values
        )

# Create ps_percent values from 0.5 to 1.0.
ps_values = np.linspace(0.5, 1.0, 20)

# Baseline: Free yaw with TCIPC_MaxTipDeflection=20.
baseline_aep = []
baseline_clearance = []
for ps in ps_values:
    baseline_aep.append(interpolators["Free yaw"]["aep"](ps, 20))
    baseline_clearance.append(interpolators["Free yaw"]["tower_clearance"](ps, 20))

# Free yaw: Free yaw with TCIPC_MaxTipDeflection=0.
free_yaw_aep = []
free_yaw_clearance = []
for ps in ps_values:
    free_yaw_aep.append(interpolators["Free yaw"]["aep"](ps, 0))
    free_yaw_clearance.append(interpolators["Free yaw"]["tower_clearance"](ps, 0))

# Zero yaw: Zero yaw with TCIPC_MaxTipDeflection=0.
zero_yaw_aep = []
zero_yaw_clearance = []
for ps in ps_values:
    zero_yaw_aep.append(interpolators["Zero yaw"]["aep"](ps, 0))
    zero_yaw_clearance.append(interpolators["Zero yaw"]["tower_clearance"](ps, 0))

# Create the plot.
fig, ax = plt.subplots(
    figsize=(
        plt.rcParams["figure.figsize"][0] * 0.8,
        2.25,
    )
)

# Plot lines for each controller.
line1 = ax.plot(
    -np.array(baseline_clearance),
    -np.array(baseline_aep),
    label="Baseline",
)[0]
line2 = ax.plot(
    -np.array(free_yaw_clearance),
    -np.array(free_yaw_aep),
    label="Free yaw",
)[0]
line3 = ax.plot(
    -np.array(zero_yaw_clearance),
    -np.array(zero_yaw_aep),
    label="Zero yaw",
)[0]

# Add scatter points for 50% peak shaving (first point, empty markers).
ax.scatter(
    -np.array(baseline_clearance)[0],
    -np.array(baseline_aep)[0],
    color=line1.get_color(),
    s=50,
    facecolors="none",
    edgecolors=line1.get_color(),
    zorder=3,
)
ax.scatter(
    -np.array(free_yaw_clearance)[0],
    -np.array(free_yaw_aep)[0],
    color=line2.get_color(),
    s=50,
    facecolors="none",
    edgecolors=line2.get_color(),
    zorder=3,
)
ax.scatter(
    -np.array(zero_yaw_clearance)[0],
    -np.array(zero_yaw_aep)[0],
    color=line3.get_color(),
    s=50,
    facecolors="none",
    edgecolors=line3.get_color(),
    zorder=3,
)

# Add scatter points for 0% peak shaving (last point, filled markers).
ax.scatter(
    -np.array(baseline_clearance)[-1],
    -np.array(baseline_aep)[-1],
    color=line1.get_color(),
    s=50,
    zorder=3,
)
ax.scatter(
    -np.array(free_yaw_clearance)[-1],
    -np.array(free_yaw_aep)[-1],
    color=line2.get_color(),
    s=50,
    zorder=3,
)
ax.scatter(
    -np.array(zero_yaw_clearance)[-1],
    -np.array(zero_yaw_aep)[-1],
    color=line3.get_color(),
    s=50,
    zorder=3,
)

# Create custom legend entries for peak shaving levels.
legend_elements = [
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="black",
        label="0% peak shaving",
        markersize=6,
        linestyle="",
    ),
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="black",
        label="50% peak shaving",
        markersize=6,
        linestyle="",
        markerfacecolor="none",
    ),
]

# Create two legends: one for controllers, one for peak shaving levels.
legend1 = ax.legend(loc="upper right")
ax.add_artist(legend1)
ax.legend(handles=legend_elements, loc="upper left")

ax.set_xlabel("Tower clearance (m)")
ax.set_ylabel("AEP (GWh)")
ax.grid()

# ax.set_ylim((10, 25))
ax.set_ylim((70, 95))
ax.set_xlim((11, 26))

plt.tight_layout()
plt.savefig("../figures/trade-off.pdf")
plt.show()


In [ ]:
(90 - 71.75) / 90

In [ ]:
# Plot 4: Alternative trade-off comparison.
# Baseline: 20% peak shaving with TCIPC=20 (single point).
# Peak shaving: vary peak shaving 0-50% with TCIPC=0.
# Free yaw: 20% peak shaving with TCIPC 0-20m.
# Zero yaw: 20% peak shaving with TCIPC 0-20m.

# Create values for the lines.
ps_values_sweep = np.linspace(0.5, 0.8, 20)  # 50% to 0% peak shaving
tcipc_values_sweep = np.linspace(0, 20, 20)  # 0 to 20m TCIPC reference
ps_baseline = 0.8  # 20% peak shaving

# Baseline: 20% peak shaving, TCIPC=20m (single point).
baseline_aep_alt = interpolators["Free yaw"]["aep"](ps_baseline, 20)
baseline_clearance_alt = interpolators["Free yaw"]["tower_clearance"](ps_baseline, 20)

# Peak shaving: vary ps from 50% to 0%, TCIPC=20.
peak_shaving_aep = []
peak_shaving_clearance = []
for ps in ps_values_sweep:
    peak_shaving_aep.append(interpolators["Free yaw"]["aep"](ps, 20))
    peak_shaving_clearance.append(interpolators["Free yaw"]["tower_clearance"](ps, 20))

# Free yaw: 20% peak shaving, TCIPC 0-20m.
free_yaw_aep_alt = []
free_yaw_clearance_alt = []
for tcipc in tcipc_values_sweep:
    free_yaw_aep_alt.append(interpolators["Free yaw"]["aep"](ps_baseline, tcipc))
    free_yaw_clearance_alt.append(
        interpolators["Free yaw"]["tower_clearance"](ps_baseline, tcipc)
    )

# Zero yaw: 20% peak shaving, TCIPC 0-20m.
zero_yaw_aep_alt = []
zero_yaw_clearance_alt = []
for tcipc in tcipc_values_sweep:
    zero_yaw_aep_alt.append(interpolators["Zero yaw"]["aep"](ps_baseline, tcipc))
    zero_yaw_clearance_alt.append(
        interpolators["Zero yaw"]["tower_clearance"](ps_baseline, tcipc)
    )

# Create the plot.
fig, ax = plt.subplots(figsize=(plt.rcParams["figure.figsize"][0] * 0.8, 3 * 0.8))

# Plot baseline as a single point.
ax.scatter(
    -baseline_clearance_alt,
    -baseline_aep_alt,
    s=50,
    color="black",
    label="Baseline",
    zorder=4,
)

# Plot lines for each strategy.
line1 = ax.plot(
    -np.array(peak_shaving_clearance),
    -np.array(peak_shaving_aep),
    label="Peak shaving",
)[0]
line2 = ax.plot(
    -np.array(free_yaw_clearance_alt),
    -np.array(free_yaw_aep_alt),
    label="Free yaw",
)[0]
line3 = ax.plot(
    -np.array(zero_yaw_clearance_alt),
    -np.array(zero_yaw_aep_alt),
    label="Zero yaw",
)[0]


# Create two legends: one for strategies, one for endpoints.
ax.legend(loc="lower left")

ax.set_xlabel("Tower clearance (m)")
ax.set_ylabel("AEP (GWh)")
ax.grid()

# ax.set_ylim((70, 95))
# ax.set_xlim((11, 26))

plt.tight_layout()
# plt.savefig("../figures/trade-off-alternative.pdf")
plt.show()

## Optimization

In [ ]:
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.algorithms.moo.ctaea import CTAEA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from scipy.interpolate import CloughTocher2DInterpolator


# Create separate interpolators for each log
interpolators = {}
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
    "Ct",
]
signs = [-1, -1, +1, +1, +1, +1, +1]
for log_name in all_data_dicts.keys():
    df_log = df[df["log_name"] == log_name]
    xy = df_log[["ps_percent", "TCIPC_MaxTipDeflection"]].values

    interpolators[log_name] = {}

    for j, output in enumerate(outputs):
        interpolators[log_name][output] = CloughTocher2DInterpolator(
            xy, signs[j] * df_log[output].values
        )


class TuningProblem(ElementwiseProblem):
    def __init__(self, objective_functions, inequaltiy_constraints, xl=None, xu=None):
        # Set some defaults.
        if xl is None:
            xl = np.array([0.5, 0.0])
        if xu is None:
            xu = np.array([1.0, 20.0])

        super().__init__(
            n_var=2,
            n_obj=len(objective_functions),
            n_ieq_constr=len(inequaltiy_constraints),
            xl=xl,
            xu=xu,
        )

        self.objective_functions = objective_functions
        self.inequaltiy_constraints = inequaltiy_constraints

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = [f(*x) for f in self.objective_functions]
        out["G"] = [g(*x) for g in self.inequaltiy_constraints]


# Create problems for baseline, free yaw, and zero yaw deflection
# Baseline: TCIPC_MaxTipDeflection=20 (TCIPC disabled), ps_percent is free
# Use Free yaw data for baseline (could use either dataset)
DEL_RootMyb_baseline = interpolators["Free yaw we0"]["DEL_RootMyb"](0.8, 20)
Ct_baseline = interpolators["Free yaw we0"]["Ct"](0.8, 20)


problem_baseline = TuningProblem(
    [
        interpolators["Free yaw we0"]["aep"],
        interpolators["Free yaw we0"]["tower_clearance"],
    ],
    [
        # lambda x, y: interpolators["Free yaw we0"]["DEL_RootMyb"](x, y)
        # - DEL_RootMyb_baseline,
        lambda x, y: interpolators["Free yaw we0"]["Ct"](x, y) - Ct_baseline,
    ],
    xl=np.array([0.5, 20.0]),
    xu=np.array([1.0, 20.0]),  # Fixed TCIPC, free ps_percent
)

# Free yaw: full range
problem_free_yaw = TuningProblem(
    [
        interpolators["Free yaw we0"]["aep"],
        interpolators["Free yaw we0"]["tower_clearance"],
    ],
    [
        # lambda x, y: interpolators["Free yaw we0"]["DEL_RootMyb"](x, y)
        # - DEL_RootMyb_baseline,
        lambda x, y: interpolators["Free yaw we0"]["Ct"](x, y) - Ct_baseline,
    ],
)

# Zero yaw deflection: full range
problem_zero_yaw = TuningProblem(
    [
        interpolators["Zero yaw we0"]["aep"],
        interpolators["Zero yaw we0"]["tower_clearance"],
    ],
    [
        # lambda x, y: interpolators["Zero yaw we0"]["DEL_RootMyb"](x, y)
        # - DEL_RootMyb_baseline,
        lambda x, y: interpolators["Zero yaw we0"]["Ct"](x, y) - Ct_baseline,
    ],
)

algorithm = NSGA2(
    pop_size=150,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)
# ref_dirs = get_reference_directions("uniform", 2, n_partitions=50)
# algorithm = MOEAD(
#     ref_dirs,
#     n_neighbors=50,
#     prob_neighbor_mating=0.7,
# )
# algorithm = CTAEA(ref_dirs)  # I found this one to work very well.


termination = get_termination("n_gen", 100)

# Run optimization for all three cases
result_baseline = minimize(
    problem_baseline, algorithm, termination, seed=1, save_history=True, verbose=True
)
result_free_yaw = minimize(
    problem_free_yaw, algorithm, termination, seed=1, save_history=True, verbose=True
)
result_zero_yaw = minimize(
    problem_zero_yaw, algorithm, termination, seed=1, save_history=True, verbose=True
)

In [ ]:
# Analyze convergence for all three cases
from pymoo.indicators.hv import Hypervolume


# Function to calculate hypervolume history for a result
def calculate_hv_history(result):
    n_evals = []
    hist_F = []
    hist_cv = []
    hist_cv_avg = []

    for algo in result.history:
        n_evals.append(algo.evaluator.n_eval)
        opt = algo.opt
        hist_cv.append(opt.get("CV").min())
        hist_cv_avg.append(algo.pop.get("CV").mean())
        feas = np.where(opt.get("feasible"))[0]
        hist_F.append(opt.get("F")[feas])

    approx_ideal = result.F.min(axis=0)
    approx_nadir = result.F.max(axis=0)
    ref_point = approx_nadir

    metric = Hypervolume(
        ref_point=ref_point,
        norm_ref_point=False,
        zero_to_one=False,
        ideal=approx_ideal,
        nadir=approx_nadir,
    )

    hv = [metric.do(_F) for _F in hist_F]
    return hv, n_evals


# Calculate hypervolume for all cases
hv_baseline, n_evals_baseline = calculate_hv_history(result_baseline)
hv_free_yaw, n_evals_free_yaw = calculate_hv_history(result_free_yaw)
hv_zero_yaw, n_evals_zero_yaw = calculate_hv_history(result_zero_yaw)

# Plot convergence for all three cases
plt.figure(figsize=(7, 5))
plt.plot(hv_baseline, "o-", lw=1.5, label="Baseline", markersize=4)
plt.plot(hv_free_yaw, "s-", lw=1.5, label="Free Yaw", markersize=4)
plt.plot(hv_zero_yaw, "^-", lw=1.5, label="Zero Yaw", markersize=4)
plt.title("Convergence Comparison")
plt.xlabel("Generation")
plt.ylabel("Hypervolume")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Pareto front comparison - all three cases
pareto_baseline = -result_baseline.F[result_baseline.F[:, 0].argsort()]  # type: ignore
pareto_free_yaw = -result_free_yaw.F[result_free_yaw.F[:, 0].argsort()]  # type: ignore
pareto_zero_yaw = -result_zero_yaw.F[result_zero_yaw.F[:, 0].argsort()]  # type: ignore

plt.figure(figsize=(7, 5))
plt.plot(
    *np.transpose(pareto_baseline),
    "o-",
    label="Baseline",
    linewidth=2,
    markersize=8,
)
plt.plot(
    *np.transpose(pareto_free_yaw),
    "s-",
    label="Free Yaw",
    linewidth=2,
    markersize=6,
)
plt.plot(
    *np.transpose(pareto_zero_yaw),
    "^-",
    label="Zero Yaw",
    linewidth=2,
    markersize=6,
)
plt.legend()
plt.xlabel("AEP (GWh)")
plt.ylabel("Tower clearance (m)")
plt.title("Pareto Front Comparison")
plt.grid(alpha=0.3)
plt.tight_layout()

In [ ]:
# Visualize design space for all three cases
xl, xu = problem_free_yaw.bounds()
plt.figure(figsize=(7, 5))
plt.scatter(
    result_baseline.X[:, 0],
    result_baseline.X[:, 1],
    s=50,
    # marker="*",
    # c="red",
    label="Baseline",
    zorder=3,
)
plt.scatter(
    result_free_yaw.X[:, 0],
    result_free_yaw.X[:, 1],
    s=30,
    # facecolors="none",
    # edgecolors="blue",
    label="Free Yaw",
)
plt.scatter(
    result_zero_yaw.X[:, 0],
    result_zero_yaw.X[:, 1],
    s=30,
    # facecolors="none",
    # edgecolors="green",
    label="Zero Yaw",
)
plt.xlim(xl[0], xu[0])
plt.ylim(xl[1], xu[1])
plt.xlabel("Peak Shaving (%)")
plt.ylabel("TCIPC Max Tip Deflection (m)")
plt.title("Design Space - All Cases")
plt.legend()
plt.grid(alpha=0.3)